In [1]:
#%pip install pandas
import pandas as pd # type: ignore
import datetime
import numpy as np # type: ignore

In [8]:
# dependencies
%pip install pyarrow
%pip install fastparquet

   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.2 MB 325.1 kB/s eta 0:01:18
   ---------------------------------------- 0.1/25.2 MB 762.6 kB/s eta 0:00:33
    --------------------------------------- 0.4/25.2 MB 2.8 MB/s eta 0:00:09
   - -------------------------------------- 0.9/25.2 MB 4.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.6/25.2 MB 6.3 MB/s eta 0:00:04
   --- ------------------------------------ 2.3/25.2 MB 7.7 MB/s eta 0:00:03
   ---- ----------------------------------- 3.1/25.2 MB 8.8 MB/s eta 0:00:03
   ------ --------------------------------- 3.9/25.2 MB 10.5 MB/s eta 0:00:03
   ------- -------------------------------- 4.8/25.2 MB 11.4 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/25.2 MB 11.9 MB/s eta 0:00:02
   -------- -


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/672.0 kB ? eta -:--:--
    --------------------------------------- 10.2/672.0 kB ? eta -:--:--
   -- ------------------------------------ 41.0/672.0 kB 393.8 kB/s eta 0:00:02
   -------- ------------------------------- 143.4/672.0 kB 1.1 MB/s eta 0:00:01
   ----------------------- ---------------- 389.1/672.0 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 672.0/672.0 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------- -------------------------- 0.7/2.1 MB 14.8 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 26.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 19.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/179.3 kB ? eta -:--:--
   --------------------------------------- 179.3/179.3 kB 11.3 MB/s eta 0:00:00


In [2]:
year_list = list(map(str, range(2000, 2020)))

In [5]:
# function - delete column - takes df and list of column names in str
def delete_col(target_df, target_col):
    target_df.drop(target_col, axis=1, inplace=True)
    return target_df

# function - change date to year in the df - takes df
def to_year(target_df):
    if 'date' in target_df.columns: target_df['date'] = pd.to_datetime(target_df['date'])
    target_df['date'] = target_df['date'].dt.strftime('%Y')
    target_df['date'] = target_df['date'].astype(int)
    return target_df

# function - change columns names - takes df and list of column names in str
def change_col_names(target_df, names_list):
    target_df.columns = names_list
    return target_df

In [46]:
URL_DATA = 'https://storage.dosm.gov.my/population/population_malaysia.parquet'

df_population = pd.read_parquet(URL_DATA)
df_population = df_population[(df_population['sex'] == 'both') &
                              (df_population['ethnicity'] == 'overall') &
                              (df_population['age'] != 'overall')]
df_population = delete_col(df_population, ['sex', 'ethnicity'])
df_population = to_year(df_population)
df_population = change_col_names(df_population, ['Year', 'Age', 'Population'])

df_population_00_22 = df_population[df_population['Year'] > 1999]
df_population_00_22 = df_population_00_22[df_population_00_22['Year'] < 2023].reset_index()
df_population_00_22.drop('index', axis=1, inplace=True)

df_population_00_22.tail()

,Year,Age,Population
409,2022,65-69,974.5
410,2022,70-74,674.5
411,2022,75-79,377.1
412,2022,80-84,203.1
413,2022,85+,134.5


In [47]:
# population of fertile adults (20 - 40 age)
df_population_00_22_fertile_adult = df_population_00_22[(df_population_00_22['Age'] == '20-24') |
                                                        (df_population_00_22['Age'] == '25-29') |
                                                        (df_population_00_22['Age'] == '30-34') |
                                                        (df_population_00_22['Age'] == '35-39')]
df_population_00_22_fertile_adult = df_population_00_22_fertile_adult.groupby(['Year'])['Population'].sum().reset_index()
df_population_00_22_fertile_adult = change_col_names(df_population_00_22_fertile_adult, ['Year', 'Number of Fertile Adults (Age 20 - 39, x 1000)'])
df_population_00_22_fertile_adult.tail()

,Year,"Number of Fertile Adults (Age 20 - 39, x 1000)"
18,2018,11821.9
19,2019,11834.6
20,2020,11408.4
21,2021,11388.1
22,2022,11374.9


In [48]:
URL_DATA = 'https://storage.dosm.gov.my/demography/birth.parquet'

df_birth = pd.read_parquet(URL_DATA)
df_birth = to_year(df_birth)
df_birth = change_col_names(df_birth, ['Year', 'Birth', 'Birth Rate'])

df_birth_00_22 = df_birth[df_birth['Year'] > 1999]
df_birth_00_22 = df_birth_00_22[df_birth_00_22['Year'] < 2023].reset_index()
df_birth_00_22.drop('index', axis=1, inplace=True)

df_birth_00_22.head()

,Year,Birth,Birth Rate
0,2000,537853,22.9
1,2001,505479,21.0
2,2002,494538,20.2
3,2003,481399,19.2
4,2004,481800,18.9


In [50]:
URL_DATA = 'https://storage.dosm.gov.my/demography/marriages.parquet'

df_marriage = pd.read_parquet(URL_DATA)
df_marriage = to_year(df_marriage)
df_marriage = change_col_names(df_marriage, ['Year', 'Sex', 'Marriage Number', 'Marriage Rate'])
df_marriage = delete_col(df_marriage, ['Sex', 'Marriage Rate'])
df_marriage.drop_duplicates(subset='Marriage Number', inplace=True)

df_marriage_00_22 = df_marriage[df_marriage['Year'] > 1999].reset_index()
#df_marriage_00_22 = df_marriage_00_20[df_marriage_00_20['Year'] < 2023].reset_index()
df_marriage_00_22.drop('index', axis=1, inplace=True)

df_marriage_00_22.tail()

,Year,Marriage Number
1,2018,206352
2,2019,203661
3,2020,184589
4,2021,215973
5,2022,214824


In [52]:
URL_DATA = 'https://storage.dosm.gov.my/demography/fertility.parquet'

df_fertility = pd.read_parquet(URL_DATA)

df_fertility = to_year(df_fertility)

df_fertility = df_fertility[df_fertility['age_group'] == 'tfr']
df_fertility = delete_col(df_fertility, 'age_group')
df_fertility = change_col_names(df_fertility, ['Year', 'Fertility Rate'])

df_fertility_00_22 = df_fertility[df_fertility['Year'] > 1999]
df_fertility_00_22 = df_fertility_00_22[df_fertility_00_22['Year'] < 2023].reset_index()
df_fertility_00_22.drop('index', axis=1, inplace=True)

df_fertility_00_22.tail()

,Year,Fertility Rate
18,2018,1.84
19,2019,1.78
20,2020,1.71
21,2021,1.70
22,2022,1.63


In [53]:
df_birth_vs_fertile_adult = pd.merge(df_birth_00_22, df_population_00_22_fertile_adult, how='inner')
df_birth_vs_fertile_adult.drop('Birth Rate', axis=1, inplace=True)
df_birth_vs_fertile_adult['Birth'] = df_birth_vs_fertile_adult['Birth'] / 1000
df_birth_vs_fertile_adult = change_col_names(df_birth_vs_fertile_adult, ['Year', 'Birth (x 1000)', 'Number of Fertile Adults (Age 20 - 39, x 1000)'])
df_birth_vs_fertile_adult.tail()

,Year,Birth (x 1000),"Number of Fertile Adults (Age 20 - 39, x 1000)"
18,2018,501.945,11821.9
19,2019,489.863,11834.6
20,2020,471.504,11408.4
21,2021,439.744,11388.1
22,2022,423.124,11374.9


In [21]:
%pip install matplotlib
%pip install seaborn

     ---------------------------------------- 0.0/167.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/167.0 kB ? eta -:--:--
     ------------------- ------------------- 81.9/167.0 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 167.0/167.0 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB 10.2 MB/s eta 0:00:01
   ---- ----------------------------------- 0.8/7.8 MB 10.1 MB/s eta 0:00:01
   ------ --------------------------------- 1.3/7.8 MB 10.0 MB/s eta 0:00:01
   -------- ------------------------------- 1.7/7.8 MB 9.2 MB/s eta 0:00:01
   -------- ------------------------------- 1.7/7.8 MB 9.2 MB/s eta 0:00:01
   ---------- ----------------------------- 2.1/7.8 MB 7.4 MB/s eta 0:00:01
   ------------- -------------------------- 2.6/7.8 MB 8.2 MB/s eta 0:00:01
   --------------- ------------------------ 2.9/7.8 MB 8.2 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.9 kB ? eta -:--:--
   ------------ --------------------------- 92.2/294.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 2.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
%pip install plotly

   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/19.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/19.1 MB 812.7 kB/s eta 0:00:24
   - -------------------------------------- 0.6/19.1 MB 5.0 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/19.1 MB 13.6 MB/s eta 0:00:02
   -------- ------------------------------- 4.3/19.1 MB 19.5 MB/s eta 0:00:01
   ------------- -------------------------- 6.4/19.1 MB 25.7 MB/s eta 0:00:01
   ----------------- ---------------------- 8.3/19.1 MB 27.8 MB/s eta 0:00:01
   ----------------------- ---------------- 11.2/19.1 MB 50.4 MB/s eta 0:00:01
   ------------------------------ --------- 14.3/19.1 MB 50.4 MB/s eta 0:00:01
   ------------------------------------- -- 17.7/19.1 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------  19.0/19.1 MB 59.8 MB/s eta 0:00:01
   ---------------------------------------- 19.1/19.1 MB 43.7 MB/s eta 0:00:


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
import plotly.graph_objects as go # type: ignore
import plotly.express as px # type: ignore
from plotly.subplots import make_subplots # type: ignore
#%matplotlib inline

In [9]:
import plotly.io as pio # type: ignore
pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

In [32]:
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\1\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [56]:
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

fig1.add_trace(go.Scatter(x=df_birth_00_22['Year'], y=df_birth_00_22['Birth'], mode='lines+markers', name='Newborn'), secondary_y=False)
fig1.add_trace(go.Scatter(x=df_fertility_00_22['Year'], y=df_fertility_00_22['Fertility Rate'], mode='lines+markers', name='Fertility Rate'), secondary_y=True)

# Add figure title
fig1.update_layout(
    title_text="<b>Newborn and Fertility Rate over Years</b>"
)

# Set x-axis title
fig1.update_xaxes(title_text="Year")

# Set y-axes titles
fig1.update_yaxes(title_text="Newborn Number", secondary_y=False, range=[0, 600000])
fig1.update_yaxes(title_text="Fertility Rate", secondary_y=True, range=[0, 3.5])

fig1.show()

In [55]:
fig2 = make_subplots(
    rows=2, cols=2,
    subplot_titles=("<b>Number of Newborn and Fertile Adult by Year</b>",
                    "<b>Marriage Registered by Year</b>",
                    "<b>Income and Living Cost by Year</b>",
                    "<b>Poverty and Income Inequality by Year</b>"))

# Number of Newborn and Fertile Adult by Year
fig2.add_trace(go.Scatter(x=df_birth_vs_fertile_adult['Year'],
                            y=df_birth_vs_fertile_adult['Number of Fertile Adults (Age 20 - 39, x 1000)'],
                            fill='tozeroy'),
                          row=1,
                          col=1)
fig2.add_trace(go.Scatter(x=df_birth_vs_fertile_adult['Year'],
                            y=df_birth_vs_fertile_adult['Birth (x 1000)'],
                            fill='tonexty'),
                          row=1,
                          col=1)

# Marriage Registered by Year
fig2.add_trace(go.Scatter(x=df_marriage_00_22['Year'],
                          y=df_marriage_00_22['Marriage Number'],
                          mode='lines+markers'),
                          row=1,
                          col=2)

# Income and Living Cost by Year

# Poverty and Income Inequality by Year


fig2.show()

In [16]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
#from project_lib import Project
#project = Project(project_id='a2e7ffc5-f1d8-43b9-b7d5-7ed528777524', project_access_token='p-2+kJVKIeETJ/Q+4uOYczkVAw==;+f1Ev4sNWIGLz3hKvGPM/Q==:VFGLUdxJCCZtNJ8sYXNtzWjEShCw3Zo3nx5SiutXusuWscm5fBM2WYVyuCayaw9Y+Dsqo44u4BKd7XRgOIxpy06MBrKl4ZN27w==')
#pc = project.project_context

#from ibm_watson_studio_lib import access_project_or_space
#wslib = access_project_or_space({'token':'p-2+kJVKIeETJ/Q+4uOYczkVAw==;+f1Ev4sNWIGLz3hKvGPM/Q==:VFGLUdxJCCZtNJ8sYXNtzWjEShCw3Zo3nx5SiutXusuWscm5fBM2WYVyuCayaw9Y+Dsqo44u4BKd7XRgOIxpy06MBrKl4ZN27w=='})


In [ ]:
figs = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]

dashboard = open("dashboard.html", "w")
dashboard.write("<html><body>" + "\n")
dashboard.write("<h>Newborn Trend in Malaysia</h>" + "\n")
for fig in figs:
    dashboard.write(fig.to_html(full_html=False, include_plotlyjs='cdn') + "\n")
dashboard.write("</body></html>")
dashboard.close()